# BG/NBD 模型 - 预测客户购买行为
> By SherryAGI | 概率模型预测未来购买次数

## BG/NBD 模型简介

**Beta-Geometric/Negative Binomial Distribution** 模型用于预测：
- 客户未来会购买多少次
- 客户是否已经流失

### 核心假设
1. 客户购买遵循泊松过程
2. 每次购买后有一定概率流失
3. 购买率和流失率因人而异

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 安装 lifetimes: pip install lifetimes
from lifetimes import BetaGeoFitter
from lifetimes.plotting import plot_frequency_recency_matrix, plot_probability_alive_matrix
from lifetimes.utils import summary_data_from_transaction_data

print('🚀 BG/NBD 购买预测模型 by SherryAGI')

## 1. 准备数据

In [ ]:
# 模拟交易数据
np.random.seed(42)
n_transactions = 10000

transactions = pd.DataFrame({
    'customer_id': np.random.randint(1, 1001, n_transactions),
    'date': pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(0, 365, n_transactions), unit='D'),
    'amount': np.random.exponential(100, n_transactions)
})

transactions = transactions.sort_values(['customer_id', 'date'])
transactions.head(10)

In [ ]:
# 转换为 RFM 格式 (lifetimes 要求)
rfm = summary_data_from_transaction_data(
    transactions,
    customer_id_col='customer_id',
    datetime_col='date',
    monetary_value_col='amount'
)

print(f'客户数: {len(rfm)}')
rfm.head(10)

### 数据说明
- **frequency**: 重复购买次数（首次购买不算）
- **recency**: 首次购买到最后一次购买的天数
- **T**: 首次购买到观察期结束的天数
- **monetary_value**: 平均订单金额

## 2. 训练 BG/NBD 模型

In [ ]:
# 初始化并训练模型
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(rfm['frequency'], rfm['recency'], rfm['T'])

print('模型参数:')
print(bgf.summary)

## 3. 预测未来购买次数

In [ ]:
# 预测未来 30 天的购买次数
t = 30  # 预测期（天）

rfm['predicted_purchases_30d'] = bgf.conditional_expected_number_of_purchases_up_to_time(
    t,
    rfm['frequency'],
    rfm['recency'],
    rfm['T']
)

# 预测未来 90 天
rfm['predicted_purchases_90d'] = bgf.conditional_expected_number_of_purchases_up_to_time(
    90,
    rfm['frequency'],
    rfm['recency'],
    rfm['T']
)

rfm.sort_values('predicted_purchases_90d', ascending=False).head(10)

## 4. 客户存活概率

In [ ]:
# 计算客户还"活着"的概率
rfm['prob_alive'] = bgf.conditional_probability_alive(
    rfm['frequency'],
    rfm['recency'],
    rfm['T']
)

# 存活概率分布
print(f'平均存活概率: {rfm["prob_alive"].mean():.2%}')
print(f'高风险客户 (存活<50%): {(rfm["prob_alive"] < 0.5).sum()} 人')

## 5. 可视化

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 频率-最近购买矩阵
plot_frequency_recency_matrix(bgf, ax=axes[0])
axes[0].set_title('预期购买次数矩阵 by SherryAGI')

# 存活概率矩阵
plot_probability_alive_matrix(bgf, ax=axes[1])
axes[1].set_title('客户存活概率矩阵')

plt.tight_layout()
plt.show()

## 6. 业务应用

| 指标 | 应用场景 |
|------|----------|
| 预测购买次数 | 销售预测、库存规划 |
| 存活概率 | 流失预警、挽回营销 |
| 高价值客户 | VIP 服务、精准营销 |

In [ ]:
# 识别需要挽回的客户
at_risk = rfm[(rfm['prob_alive'] < 0.5) & (rfm['frequency'] > 2)]
print(f'🚨 高风险流失客户（曾经活跃）: {len(at_risk)} 人')
print('建议：发送挽回邮件、专属优惠券')

---
**By SherryAGI** | [DigitalTrans](https://github.com/AIB612/DigitalTrans)